## Student #1 ID: 207781956

## Student #2 ID:

# Exercise 1: Decision Trees

In this assignment you will implement a Decision Tree algorithm as learned in class.

## Read the following instructions carefully:

1. This jupyter notebook contains all the step by step instructions needed for this exercise.
1. Write **efficient vectorized** code whenever possible. Some calculations in this exercise take several minutes when implemented efficiently, and might take much longer otherwise. Unnecessary loops will result in point deduction.
1. You are responsible for the correctness of your code and should add as many tests as you see fit. Those tests will not be graded nor checked.
1. You are free to add code and markdown cells as you see fit.
1. Write your functions in this jupyter notebook only. Do not create external python modules and import from them.
1. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/) and [numpy](https://www.numpy.org/devdocs/reference/) only, unless otherwise mentioned.
1. Your code must run without errors. It is a good idea to restart the notebook and run it from end to end before you submit your exercise.
1. Answers to qualitative questions should be written in **markdown cells (with $\LaTeX$ support)**.
1. Submit this jupyter notebook only using your ID as a filename. **No not use ZIP or RAR**. For example, your submission should look like this: `123456789.ipynb` if you worked by yourself or `123456789_987654321.ipynb` if you worked in pairs.

## In this exercise you will perform the following:
1. Practice OOP in python.
2. Implement two impurity measures: Gini and Entropy.
3. Construct a decision tree algorithm.
4. Prune the tree to achieve better results.
5. Visualize your results.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# make matplotlib figures appear inline in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Make the notebook automatically reload external python modules
%load_ext autoreload
%autoreload 2

## Warmup - OOP in python

Our desicion tree will be implemented using a dedicated python class. Take a minute and practice your object oriented skills. Create a tree with some nodes and make sure you understand how objects in python work.

In [5]:
class Node(object):
    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, node):
        self.children.append(node)

In [6]:
n = Node(5)
p = Node(6)
q = Node(7)
n.add_child(p)
n.add_child(q)
n.children

[<__main__.Node at 0x290a8887890>, <__main__.Node at 0x290ac5fcb90>]

## Data preprocessing

We will use the breast cancer dataset that is available as a part of sklearn. In this example, our dataset will be a single matrix with the **labels on the last column**. Notice that you are not allowed to use additional functions from sklearn.

In [7]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

# load dataset
X, y = datasets.load_breast_cancer(return_X_y = True)
X = np.column_stack([X,y]) # the last column holds the labels

# split dataset
X_train, X_test = train_test_split(X, random_state=99)

print("Training dataset shape: ", X_train.shape)
print("Testing dataset shape: ", X_test.shape)

all_labels =  [X_train[i][30] for i in range(30)]
all_labels

Training dataset shape:  (426, 31)
Testing dataset shape:  (143, 31)


[np.float64(1.0),
 np.float64(1.0),
 np.float64(0.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(1.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(1.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(1.0),
 np.float64(1.0)]

## Impurity Measures (10 points)

Impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. Implement the functions `calc_gini` (5 points) and `calc_entropy` (5 points). You are encouraged to test your implementation.

In [8]:
def calc_gini(data):
    """
    Calculate gini impurity measure of a dataset.
 
    Input:
    - data: any dataset where the last column holds the labels.
 
    Returns the gini impurity.    
    """
   
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    
    # Extract labels (last column)
    labels = data[:, -1]
    
    # If no data, return 0 (pure)
    if len(labels) == 0:
        return 0.0
    
    # Count occurrences of each class
    unique_labels, counts = np.unique(labels, return_counts=True)
    
    # Calculate probabilities
    probabilities = counts / len(labels)
    
    # Calculate Gini impurity: 1 - sum(p_i^2)
    gini = 1.0 - np.sum(probabilities ** 2)
    
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return gini

In [9]:
def calc_entropy(data):
    """
    Calculate the entropy of a dataset.

    Input:
    - data: any dataset where the last column holds the labels.

    Returns the entropy of the dataset.    
    """
    entropy = 0.0
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    # Extract labels (last column)
    labels = data[:, -1]
    
    # If no data, return 0 (pure)
    if len(labels) == 0:
        return 0.0
    
        
    # Count occurrences of each class
    unique_labels, counts = np.unique(labels, return_counts=True)
    
    # Calculate probabilities
    probabilities = counts / len(labels)
    
    # Calculate Entropy: - sum(p_i * log2(p_i))
    entropy = -np.sum(probabilities * np.log2(probabilities))

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return entropy


## Building a Decision Tree (50 points)

Use a Python class to construct the decision tree. Your class should support the following functionality:

1. Initiating a node for a decision tree. You will need to use several class methods and class attributes and you are free to use them as you see fit. We recommend that every node will hold the **feature** and **value** used for the split and the **children** of that node. In addition, it might be a good idea to store the **prediction** in that node, the **height** of the tree for that node and whether or not that node is a **leaf** in the tree.
2. Your code should support both Gini and Entropy as impurity measures. 
3. The provided data includes continuous data. For this exercise, create at most a **single split** for each node of the tree (your tree will be binary). Determine the threshold for splitting by checking all possible features and the values available for splitting. When considering the values, take the average of each consecutive pair. For example, for the values [1,2,3,4,5] you should test possible splits on the values [1.5, 2.5, 3.5, 4.5]. 
5. After you complete building the class for a decision node in the tree, complete the function `build_tree`. This function takes as input the training dataset and the impurity measure. Then, it initializes a root for the decision tree and constructs the tree according to the procedure you saw in class.
1. Once you are finished, construct two trees: one with Gini as an impurity measure and the other using Entropy.

In [133]:
class DecisionNode:
    '''
    This class will hold everyhing you need to construct a node in a DT. You are required to 
    support basic functionality as previously described. It is highly recommended that you  
    first read and understand the entire exercises before diving into this class.
    You are allowed to change the structure of this class as you see fit.
    '''
 
    def __init__(self, data):
        # you should take more arguments as inputs when initiating a new node
        self.data = data
        self.children = []
        self.is_leaf = True
        self.split_feature = None
        self.split_value = None
        
    def add_child(self, node):
        self.children.append(node)
     
    def check_split(self, feature, value):
        greater_child = []
        not_greater_child = []
        # this function divides the data according to a specific feature and value
        # you should use this function while testing for the optimal split
        sorted_data = self.data[self.data[:,feature].argsort()] #the data sorted by the feature column.
        for i in range(len(sorted_data)):
            if sorted_data[i,feature] > value: #TODO: optimize later with binary search or something
                greater_child = sorted_data[i:] #feature > value
                not_greater_child = sorted_data[:i] #feature <= value
                break
        return greater_child, not_greater_child
         

    
    def split(self, impurity_measure):
        # this function goves over all possible features and values and finds
        # the optimal split according to the impurity measure. Note: you can
        # send a function as an argument
        node_impurity = impurity_measure(self.data)
        best_split = {'gain': None, 'greater_child' : None, 'not_greater_child' : None, 'feature' : None, 'value' : None}
        for feature in range(len(self.data[0,:-1])):
            # extract the label and the checked feature and order it according to the feature.
            split_data = self.data[self.data[:,feature].argsort()][:, [feature, -1]]
            # taking the average of each consecutive pair.
            possible_split_values = [(split_data[i,0] + split_data[i+1,0]) / 2 for i in range(len(split_data)-1)]
            for value in possible_split_values:
                greater_child, not_greater_child = self.check_split(feature, value)
                if len(greater_child) != 0:
                    greater_child_impurity = impurity_measure(greater_child)
                else:
                    greater_child_impurity = 0
                if len(not_greater_child) != 0:
                    not_greater_child_impurity = impurity_measure(not_greater_child)
                else:
                    not_greater_child_impurity = 0
                gain = node_impurity - (len(greater_child)/len(self.data)) * greater_child_impurity - (len(not_greater_child)/len(self.data)) * not_greater_child_impurity 
                # saving the split with the smaller gain
                if best_split['gain'] == None or gain < best_split['gain']:
                    best_split['greater_child'] = greater_child
                    best_split['not_greater_child'] = not_greater_child
                    best_split['feature'] = feature
                    best_split['value'] = value
                    best_split['gain'] = gain
        self.split_feature =  best_split['feature']
        self.split_value = best_split['value']
        self.add_child(DecisionNode(best_split['greater_child']))
        self.add_child(DecisionNode(best_split['not_greater_child']))
        self.is_leaf = False

In [134]:
def recursive_build(node, impurity):
    labels = node.data[:, -1]
    if len(set(labels)) != 1:
        node.split(impurity)
        recursive_build(node.children[0], impurity)
        recursive_build(node.children[1], impurity)

In [135]:
def build_tree(data, impurity):
    """
    Build a tree using the given impurity measure and training dataset. 
    You are required to fully grow the tree until all leaves are pure. 
 
    Input:
    - data: the training dataset.
    - impurity: the chosen impurity measure. Notice that you can send a function
                as an argument in python.
 
    Output: the root node of the tree.
    """
    root = DecisionNode(data)
    recursive_build(root, impurity)
    return root


In [136]:
tree_gini = build_tree(data=X_train[:20], impurity=calc_gini) 

In [137]:
tree_gini.children[0].data

array([[1.646e+01, 2.011e+01, 1.093e+02, 8.329e+02, 9.831e-02, 1.556e-01,
        1.793e-01, 8.866e-02, 1.794e-01, 6.323e-02, 3.037e-01, 1.284e+00,
        2.482e+00, 3.159e+01, 6.627e-03, 4.094e-02, 5.371e-02, 1.813e-02,
        1.682e-02, 4.584e-03, 1.779e+01, 2.845e+01, 1.235e+02, 9.812e+02,
        1.415e-01, 4.667e-01, 5.862e-01, 2.035e-01, 3.054e-01, 9.519e-02,
        0.000e+00],
       [9.667e+00, 1.849e+01, 6.149e+01, 2.891e+02, 8.946e-02, 6.258e-02,
        2.948e-02, 1.514e-02, 2.238e-01, 6.413e-02, 3.776e-01, 1.350e+00,
        2.569e+00, 2.273e+01, 7.501e-03, 1.989e-02, 2.714e-02, 9.883e-03,
        1.960e-02, 3.913e-03, 1.114e+01, 2.562e+01, 7.088e+01, 3.852e+02,
        1.234e-01, 1.542e-01, 1.277e-01, 6.560e-02, 3.174e-01, 8.524e-02,
        1.000e+00],
       [1.345e+01, 1.830e+01, 8.660e+01, 5.551e+02, 1.022e-01, 8.165e-02,
        3.974e-02, 2.780e-02, 1.638e-01, 5.710e-02, 2.950e-01, 1.373e+00,
        2.099e+00, 2.522e+01, 5.884e-03, 1.491e-02, 1.872e-02, 9.366e-03

In [138]:
# python support passing a function as arguments to another function.
tree_gini = build_tree(data=X_train, impurity=calc_gini) 
tree_entropy = build_tree(data=X_train, impurity=calc_entropy)

## Tree evaluation (10 points)

Complete the functions `predict` and `calc_accuracy`.

After building both trees using the training set (using Gini and Entropy as impurity measures), you should calculate the accuracy on the test set and print the measure that gave you the best test accuracy. For the rest of the exercise, use that impurity measure. (10 points)

In [ ]:
def predict(node, instance):
    """
    Predict a given instance using the decision tree
 
    Input:
    - root: the root of the decision tree.
    - instance: an row vector from the dataset. 
 
    Output: the prediction of the instance.
    """
    pred = None
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    
    
    
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return pred

In [ ]:
def calc_accuracy(node, dataset):
    """
    Predict a given dataset using the decision tree
 
    Input:
    - node: a node in the decision tree.
    - dataset: the dataset on which the accuracy is evaluated
 
    Output: the accuracy of the decision tree on the given dataset (%).
    """
    accuracy = 0
    ###########################################################################
    # TODO: Implement the function.                                           #
    ###########################################################################
    
    
    
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return accuracy

## Print the tree (10 points)

Complete the function `print_tree`. Your code should do something like this (10 points):
```
[X0 <= 1],
  [X1 <= 2]
    [X2 <= 3], 
       leaf: [{1.0: 10}]
       leaf: [{0.0: 10}]
    [X4 <= 5], 
       leaf: [{1.0: 5}]
       leaf: [{0.0: 10}]
   leaf: [{1.0: 50}]
```

In [140]:
def print_tree(node, feature_names=None, indent=0):
    """
    Prints the tree similar to the example above.
    As long as the print is clear, any printing scheme will be fine
    
    Input:
    - node: a node in the decision tree.
    - feature_names: optional list of feature names, default X0, X1, ...
    - indent: current indentation (used internally for recursion)
    """
    prefix = "  " * indent  # two spaces per level

    # Leaf node
    if node.is_leaf:
        # Count labels in this leaf
        labels, counts = np.unique(node.data[:, -1], return_counts=True)
        label_count = {float(l): int(c) for l, c in zip(labels, counts)}
        print(f"{prefix}leaf: [{label_count}]")
    else:
        # Feature name
        feature_name = f"X{node.split_feature}" if feature_names is None else feature_names[node.split_feature]
        print(f"{prefix}[{feature_name} <= {node.split_value}]")

        # Left child (<= threshold)
        print_tree(node.children[0], feature_names, indent + 1)

        # Right child (> threshold)
        print_tree(node.children[1], feature_names, indent + 1)

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return

## Post pruning (20 points)

Construct a decision tree and perform post pruning: For each leaf in the tree, calculate the test accuracy of the tree assuming no split occurred on the parent of that leaf and find the best such parent (in the sense that not splitting on that parent results in the best testing accuracy among possible parents). Make that parent into a leaf and repeat this process until you are left with the root. On a single plot, draw the training and testing accuracy as a function of the number of internal nodes in the tree. Explain and visualize the results and print your tree (20 points).

In [ ]:
###########################################################################
# TODO: Implement the function.                                           #
###########################################################################



###########################################################################
#                             END OF YOUR CODE                            #
###########################################################################